In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import subprocess

import sys
from copy import copy
import scipy
import imageio
sys.path.append("../")
import itertools
from scipy.stats import norm
import matplotlib.mlab as mlab
from mvgavg import mvgavg

save_folder = "G:/DREAM/from_github/thesis/Figures/LDA/"
import seaborn as sns

from random import randrange
from joblib import wrap_non_picklable_objects


from alienlab import utils
import alienlab
from alienlab import regression_func
import pickle as pk
from alienlab import plot
from activation_experiment import activation_experiment


import skimage.registration


from useful_func import dtw, get_path, plot_warped_timeseries
from useful_func import spectrum_a, spectrum_d
from useful_func import residuals, exp_decay, get_fit, sigmoid, exp_decay_max
from useful_func import gradient_magnitude, gradient_orientation, gaussian_kernel, assign_orientation, crop_center, cconv, get_algae_im
from numpy import genfromtxt

from joblib import Parallel, delayed
from useful_func import make_svm, make_pca
from sklearn.decomposition import PCA

from mpl_toolkits.axes_grid1 import make_axes_locatable

import scipy.stats as st


import sklearn
import sklearn.discriminant_analysis
import pandas as pd


import skimage

from useful_func import fit_biexp, fit_exp, fit_monoexp

import tifffile as tiff
import random
%matplotlib widget

g = plot.PlotFigure()
g.fontsize=17



In [2]:
lda = pk.load(open("lda.pkl",'rb'))
M_lda = np.genfromtxt("M_lda.csv")
S_lda = np.genfromtxt("S_lda.csv")
RTr = np.genfromtxt("RTr.csv")

def transform_data(data):
    #print(data.shape, transform.shape)
    data = (data.T - M_lda)/S_lda
    XT = np.dot(data, RTr.T)
    
    return XT


def save_david(traces_david, labels_david, desc, name):
    print(len(traces_david))
    print(len(labels_david))
    print(len(list(desc.keys())))
    np.savetxt("data_DAVID/"+name + "_labels.csv", np.concatenate(labels_david))
    np.savetxt("data_DAVID/"+ name + "_traces.csv", np.concatenate(traces_david))

    import json

    with open("data_DAVID/"+name + "_class_match.json", "w") as outfile:
        json.dump(desc, outfile)


def prepare_plot(database2):
        fig2, ax2 = plt.subplots(1, 3, figsize = (20, 8))
        labels = database2[-1]
        data = database2[:-2]
        XT =  transform_data(data)

        lower_bounds = np.quantile(XT, 0.019, axis = 0)
        upper_bounds = np.quantile(XT, 0.99, axis = 0)

        for label in [1,2,3]:
            ind = labels == label
            X = XT[ind]
            description = all_descriptions[0][ind][0]
            plot_points(X, description, ax2, lower_bounds, upper_bounds)

        return ax2, lower_bounds, upper_bounds

In [3]:

exp_set = []
exp_description = []
exp_position = []
exp_strain = []
photoinhibition_status = []
qE_status = []
qT_status = []

#ref_folder = 'Data_for_scanpy'
ref_folder = "../"
#0:stt71
#1:wt4
#2:npq4
#3:cc124
#4:WT222


if True:

    folder = ref_folder + "/FOLDER_2022_03_10_activation_4H_ENS_stt7"
    exp_description.append("stt7_001")
    exp_set.append(activation_experiment(folder))
    exp_position.append(2)
    exp_strain.append(0)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(0)

if False: 
    folder = ref_folder + "/FOLDER_2022_07_01_stt7_act_noact"
    exp_description.append("stt7_combined_000")
    exp_set.append(activation_experiment(folder))
    exp_position.append(6)
    exp_strain.append(0)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(0)
    
if True:

    folder =  ref_folder + "/FOLDER_2022_03_10_activation_4H_ENS_stt7"
    exp_description.append("stt7 qI_000")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(0)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(0)
    



if True: 
    folder = ref_folder + "/FOLDER_2022_07_01_stt7_act_noact"
    exp_description.append("stt7_combined_010")
    exp_set.append(activation_experiment(folder))
    exp_position.append(7)
    exp_strain.append(0)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(0)

if True: 
    folder = ref_folder + "/FOLDER_2022_07_01_wt4_noact"
    exp_description.append("wt4_combined_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(6)
    exp_strain.append(0)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(0)
    

if True:

    folder =  ref_folder + "/FOLDER_2022_03_10_activation_4H_ENS_stt7"
    exp_description.append("stt7 qI_000")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(0)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(0)
    
if True:

    folder =  ref_folder + "/FOLDER_2022_03_10_activation_4H_ENS_stt7"
    exp_description.append("stt7 qI qE_010")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(0)
    photoinhibition_status.append(1)
    qE_status.append(1)
    qT_status.append(0)

#################################    

if False:

    folder =  ref_folder + "/FOLDER_2022_03_08_activation_4H_ENS_wt4"
    exp_description.append("wt4 qT_101")
    exp_set.append(activation_experiment(folder))
    exp_position.append(2)
    exp_strain.append(1)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(1)

if True: 
    folder =  ref_folder +  "/FOLDER_2022_03_08_activation_4H_ENS_wt4"
    exp_description.append("wt4 qI qT_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(1)

if True:
    folder =  ref_folder + "/FOLDER_2022_03_08_activation_4H_ENS_wt4"
    exp_description.append("wt4 qI qE qT_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)#4)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(1)
    qT_status.append(1)



##################################

if False:
    folder = "../FOLDER_2022_03_09_activation_4H_ENS_npq4"
    exp_description.append("npq4 qT_101")
    exp_set.append(activation_experiment(folder))
    exp_position.append(2)
    exp_strain.append(2)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(1)

if False:
    folder = "../FOLDER_2022_03_09_activation_4H_ENS_npq4"
    exp_description.append("npq4 qI qT_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(2)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(1)

if False:
    folder = "../FOLDER_2022_03_09_activation_4H_ENS_npq4"
    exp_description.append("npq4 qI qE qT_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(2)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(1)
    
    
#######################
if True:
    folder = "../FOLDER_2022_03_16_WT222_SC1"
    exp_description.append("wt222 qT_101")
    exp_set.append(activation_experiment(folder))
    exp_position.append(2)
    exp_strain.append(1)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(1)

if True:
    folder = "../FOLDER_2022_03_16_WT222_SC1"
    exp_description.append("wt222 qI qT_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(1)

if True:
    folder = "../FOLDER_2022_03_16_WT222_SC1"
    exp_description.append("wt222 qI qE qT_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(1)
    qT_status.append(1)
    
if True:
    folder = "../FOLDER_2022_03_15_WT222_OLD"
    exp_description.append("wt222 OLD qI qE qT_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(4)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(1)
    qT_status.append(1)




######### slow activation qE

if True:
    folder = "../FOLDER_2022_02_22_HL_chunks_WT4_backup"
    exp_description.append("wt4-2H ENS not activated_000")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(1)
    qT_status.append(1)
    
if True:
    folder = "../WT4_mix"
    exp_description.append("wt4-mix")
    exp_set.append(activation_experiment(folder))
    exp_position.append(3)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(1)
    qT_status.append(1)

    #########################
if True:
    folder = "../FOLDER_2022_03_22_CC_124_activated_IBPC_sync"
    exp_description.append("CC_124 activated IBPC_110_")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = "../FOLDER_2022_03_23_npq4_activated_IBPC_sync"
    exp_description.append("npq4 IBPC_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = "../FOLDER_2022_03_24_stt71_activated_IBPC_sync"
    exp_description.append("stt71 IBPC_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)


    folder = "../FOLDER_2022_03_25_wt4_activated_IBPC_sync"
    exp_description.append("wt4 IBPC_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = "../FOLDER_2022_03_31_STT71_polyculture_slow_activateion_45min"
    exp_description.append("stt7-45min ENS_000")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)

    folder = "../FOLDER_2022_02_22_HL_chunks_WT4_backup"
    exp_description.append("wt4-2H ENS not activated_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)

    folder = "../FOLDER_2022_02_22_HL_chunks_WT4_backup"
    exp_description.append("wt4-2H ENS activated_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = "../FOLDER_2022_04_05_WT4_monoclonal_synchro_ENS_4H"
    exp_description.append("wt4-4H-monoclonal_synchro_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = "../FOLDER_2022_04_06_stt71_scl1_monoclonal_synchro_ENS_4H"
    exp_description.append("stt7-4H-monoclonal_synchro_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = "../FOLDER_2022_04_07_stt71_scl3_monoclonal_synchro_ENS_4H"
    exp_description.append("stt7-scl3-4H-monoclonal_synchro_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)



    folder = "../FOLDER_2022_03_29_4H_ENS_CC_124_sync"
    exp_description.append("CC_124_ENS_test1_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(4)
    exp_strain.append(1)

    folder = "../FOLDER_2022_03_29_4H_ENS_CC_124_sync"
    exp_description.append("CC_124_ENS_test1_noact_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)


    folder = "../FOLDER_2022_03_29_4H_ENS_CC_124_sync"
    exp_description.append("CC_124_ENS_test1_noqInoact_101")
    exp_set.append(activation_experiment(folder))
    exp_position.append(2)
    exp_strain.append(1)



    folder = "../FOLDER_2022_03_30_4H_ENS_CC_124_sync"
    exp_description.append("CC_124_ENS_test2_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)

    folder = '../FOLDER_2022_06_15_wt4_no_sync_random'
    exp_description.append("WT4 not sync_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    
    
    folder = '../FOLDER_2022_02_16_WT4_non_induit'
    exp_description.append("WT4 slow_ind_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)
    
    
    folder = '../FOLDER_2022_02_26_activation_4H_ENS_mix'
    exp_description.append("mix long_activation_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    
    
    folder = '../FOLDER_2022_02_24_HL_chunks_stt71'
    exp_description.append("npq4_chuncks_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    
    
    folder = '../FOLDER_2022_02_23_HL_chunks_npq4'
    exp_description.append("stt7-1_chuncks_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    
    
    folder = "../FOLDER_2022_03_09_activation_4H_ENS_npq4"
    exp_description.append("npq4 4H activation_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(2)
    photoinhibition_status.append(0)
    qE_status.append(0)
    qT_status.append(1)

    
if True:
    folder = "../FOLDER_2022_03_16_WT222_SC1"
    exp_description.append("wt222 qI qT_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(5)
    exp_strain.append(1)
    photoinhibition_status.append(1)
    qE_status.append(0)
    qT_status.append(1)

    
    
    folder = "../FOLDER_2022_03_30_4H_ENS_CC_124_sync"
    exp_description.append("CC_124_ENS_test2_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(5)
    exp_strain.append(1)
   

    folder = "../FOLDER_2022_07_07_CC124_not_sync_2H"
    exp_description.append("CC_124_0H_100")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)

    folder = "../FOLDER_2022_07_07_CC124_not_sync_2H"
    exp_description.append("CC_124_2H_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    
    
    folder = "../FOLDER_2022_07_07_CC124_not_sync_2H"
    exp_description.append("CC_124_4H_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(5)
    exp_strain.append(1)

        
    folder = "../FOLDER_2022_07_08_CC124_1_3h"
    exp_description.append("CC_124_1_3H_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(0)
    exp_strain.append(1)
    
            
    folder = "../FOLDER_2022_07_08_CC124_1_3h"
    exp_description.append("CC_124_1_3H_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(1)
    exp_strain.append(1)
    
            
    folder = "../FOLDER_2022_07_08_CC124_1_3h"
    exp_description.append("CC_124_1_3H_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(5)
    exp_strain.append(1)
    
    folder = "../FOLDER_2022_07_08_CC124_1_3h"
    exp_description.append("CC_124_1_3H_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(8)
    exp_strain.append(1)
    
    folder = "../FOLDER_2022_07_08_CC124_1_3h"
    exp_description.append("CC_124_1_3H_15_110")
    exp_set.append(activation_experiment(folder))
    exp_position.append(9)
    exp_strain.append(1)

In [1]:
def get_traces_from_couples(list_of_couples, traces):
    v0 = np.expand_dims(traces[:,250], 1)
    traces_0 = (v0 - traces[:,list_of_couples[0]])/traces[:,list_of_couples[0]]
    traces_1 = (v0 - traces[:,list_of_couples[1]])/traces[:,list_of_couples[1]]
    traces = traces_1 - traces_0
    return traces


def get_couple_of_measure(exps):
    liste_pair = list(itertools.combinations(np.linspace(250, 2050, 91).astype(int), 2))
    traces, labels, times = elem.get_traces_and_times(exps)

    traces = get_traces_from_couples(list_of_couples, traces)
    return traces

In [2]:
if True:
    liste_pair = list(itertools.combinations(np.linspace(250, 2050, 91).astype(int), 2))
    #liste_pair = list(itertools.combinations(np.linspace(250, 250 + 10*20, 11).astype(int), 2))
    #liste_pair = list(itertools.combinations([250, 270, 290, 310, 330, 350, 370, 390, 410, 430, 450, 470, 490, 510, 1090, 1110,  1130, 1150, 1170, 1190, 1210, 1230, 1250, 1290, 2010, 2030, 2050], 2))
    list_of_couples = list(zip(*liste_pair))
    traces_qE_set = []
    trace_set = []
    class_set = []
    strain_set  = []
    description_set = []
    couple_set = []
    actinic_set = []
    times_set = []
    pulse_set = []
    david_set = []

    
    for j, elem in enumerate(exp_set):
        
        if exp_position[j] == 0:
            exps = elem.measure[0][-2:]
            

        if exp_position[j] == 2:
            exps = elem.measure[0][0:1]

        if exp_position[j] == 1:
            exps = elem.measure[1][-2:]
        if exp_position[j]==3:
            exps = elem.measure[1]
        if exp_position[j]==4:
            exps = elem.measure[1:3][-1]
        if exp_position[j]==5:
            exps = elem.measure[2][-1:]      
        if exp_position[j]==6:
            exps = elem.measure[0]
        if exp_position[j]==7:
            exps = elem.measure[1]     
        if exp_position[j]==8:
            exps = elem.measure[3][-1:]     
        if exp_position[j]==9:
            exps = elem.measure[4][-1:] 
        traces, labels, times = elem.get_traces_and_times(exps)
        
        L =  len(traces[1])
        ints = np.linspace(0, L-1, L).astype(int)
        ints = ints[(ints-250)%20!=0]
        david_set.append(traces)

        actinic_set.append(traces[:,ints])
        pulse_set.append(traces[:,250::20])
        times_set.append(times[:,ints])
        t_qE = traces[:,250::20].T
        t_qE = (t_qE[0]- t_qE)/t_qE
        #t_qE = t_qE/t_qE[0]
        traces_qE_set.append(t_qE.T)
        traces = get_traces_from_couples(list_of_couples, traces)
        trace_set.append(traces)
        class_set.append([j]*len(traces))
        strain_set.append([exp_strain[j]]*len(traces))
        description_set.append([exp_description[j]]*len(traces))
        diff  = np.array(len(list_of_couples[0])*["-"])
        A = np.array(list_of_couples[1]).astype(str)
        B = np.array(list_of_couples[0]).astype(str)   
        A = np.char.add(A, diff)
        couple_set=np.char.add(A, B)
   

NameError: name 'itertools' is not defined

In [6]:
if False:
    np.savetxt("traces.csv", np.concatenate(david_set))
    np.savetxt("labels.csv", np.concatenate(class_set))

    desc = {}
    import json

    for i in range( len(exp_description)):
        desc[i] = exp_description[i]


    with open("class_match.json", "w") as outfile:
        json.dump(desc, outfile)


In [7]:
traces_qE = np.concatenate(traces_qE_set)
all_data = np.concatenate(trace_set, axis = 0)
all_actinic = np.concatenate(actinic_set, axis = 0)
times_set = np.concatenate(times_set)
all_descriptions = np.concatenate(description_set)
all_classes = np.concatenate(class_set)
all_strains = np.concatenate(strain_set)

indices2 = np.linspace(0, len(all_classes)-1, len(all_classes)).astype(int)

all_classes = np.expand_dims(all_classes, 0)
all_strains = np.expand_dims(all_strains, 0)
all_descriptions = np.expand_dims(all_descriptions, 0)
database = all_data
database2 = np.concatenate([database.T, all_strains, all_classes], axis = 0)

In [8]:
def plot_points(X, description, ax2, lower_bounds, upper_bounds):
    
    select = abs(X - np.mean(X, axis = 0)) < 3 * np.std(X, axis = 0)
    select = select.min(axis = 1)

    ax2[0].scatter(X[select,0], X[select,1], label = description)
    ax2[0].scatter(np.mean(X[select,0]), np.mean(X[select,1]), color = "k")

    ax2[0].set_xlabel('x - qT', fontsize=17)
    ax2[0].set_ylabel('y - qE', fontsize=17)
    ax2[0].set_xlim(lower_bounds[0], upper_bounds[0])
    ax2[0].set_ylim(lower_bounds[1], upper_bounds[1])

    ax2[1].scatter(X[select,0], X[select,2], label = description)
    ax2[1].scatter(np.mean(X[select,0]), np.mean(X[select,2]), color = "k")

    ax2[1].set_xlabel('x - qT', fontsize=17)
    ax2[1].set_ylabel('z - qI', fontsize=17)
    ax2[1].set_xlim(lower_bounds[0], upper_bounds[0])
    ax2[1].set_ylim(lower_bounds[2], upper_bounds[2])

    ax2[2].scatter(X[select,1], X[select,2], label = description)
    ax2[2].scatter(np.mean(X[select,1]), np.mean(X[select,2]), color = "k")

    ax2[2].set_xlabel('y - qE', fontsize=17)
    ax2[2].set_ylabel('z - qI', fontsize=17)
    ax2[2].set_xlim(lower_bounds[1], upper_bounds[1])
    ax2[2].set_ylim(lower_bounds[2], upper_bounds[2])

    ax2[0].legend(prop = {"size": 15})
    ax2[0].axis("off")
    ax2[1].legend(prop = {"size": 15})
    ax2[2].legend(prop = {"size": 15})

    x = X[select,0]
    y =  X[select,1]

    # Define the borders
    deltaX = (max(x) - min(x))/10
    deltaY = (max(y) - min(y))/10
    xmin = min(x) - deltaX
    xmax = max(x) + deltaX
    ymin = min(y) - deltaY
    ymax = max(y) + deltaY

    # Create meshgrid
    xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values)

    f = np.reshape(kernel(positions).T, xx.shape)


    #plt.imshow(z)

    cfset = ax2[0].contour(xx, yy, f, levels=1, alpha=1)
    #ax2[0].imshow(np.rot90(f), cmap='coolwarm', extent=[xmin, xmax, ymin, ymax])
    #cset = ax.contour(xx, yy, f, colors='k')
    #ax.clabel(cset, inline=1, fontsize=10)


## CC_124 activation 1.3h then 2H

In [28]:
labels_david = []
traces_david = []
desc = {}
k = 0
for i in range(23):
    if i!=4 and i!=22:
        ax2, lower_bounds, upper_bounds = prepare_plot(database2)

        folder = "../FOLDER_2022_07_08_CC124_1_3h"
        elem = activation_experiment(folder)
        traces, labels, times = elem.get_traces_and_times(elem.measure[i][-2:])
        traces_david.append(traces)

        if i<6:
            labels_david.append(np.array([k]*traces.shape[0]))
            desc[k] = "CC_124_activated_%dH_%dmin"%((80*k)//60, (80*k)%60)
        else:
            dur = 80*5 + 120*(k-5)
            labels_david.append(np.array([k]*traces.shape[0]))
            desc[k] = "CC_124_activated_%dH_%dmin"%(dur//60, dur%60)

        
        traces = get_couple_of_measure(elem.measure[i][-2:])
        traces = transform_data(traces.T)
        plot_points(traces, desc[k], ax2, lower_bounds, upper_bounds)
        plt.savefig(save_folder + "/CC124_long_term/" + desc[k] + ".png")
        k+= 1
        
plt.close("all")

C:\Users\Alienor\AppData\Local\Temp\ipykernel_22796\1377820401.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig2, ax2 = plt.subplots(1, 3, figsize = (20, 8))


In [10]:
name = "CC_124_short_activation"

save_david(traces_david, labels_david, desc, name)



21
21
21


## CC_124 2H activation

In [10]:
labels_david = []
traces_david = []
desc = {}
k = 0
for i in range(3):
        ax2, lower_bounds, upper_bounds = prepare_plot(database2)


        folder = "../FOLDER_2022_07_07_CC124_not_sync_2H"
        elem = activation_experiment(folder)
        traces, labels, times = elem.get_traces_and_times(elem.measure[i][-2:])
        traces_david.append(traces)
        labels_david.append(np.array([k]*traces.shape[0]))
        desc[k] = "CC_124_activated_%dH_%dmin"%((80*k)//60, (120*k)%60)


        traces = get_couple_of_measure(elem.measure[i][-2:])
        traces = transform_data(traces.T)
        plot_points(traces, desc[k], ax2, lower_bounds, upper_bounds)
        plt.savefig(save_folder + "/CC124_2H/" + desc[k] + ".png")
        k+= 1
        
plt.close("all")

In [12]:
save_folder

'G:/DREAM/from_github/thesis/Figures/LDA/'

In [13]:
name = "CC_124_2H"

save_david(traces_david, labels_david, desc, name)

3
3
3


## WT4 long activation 2H 

In [14]:
labels_david = []
traces_david = []
desc = {}
k = 0
name = "WT4_2H"

for i in range(3):
        ax2, lower_bounds, upper_bounds = prepare_plot(database2)


        folder = "../FOLDER_2022_02_22_HL_chunks_WT4_backup"
        elem = activation_experiment(folder)
        traces, labels, times = elem.get_traces_and_times(elem.measure[i][-2:])
        traces_david.append(traces)
        labels_david.append(np.array([k]*traces.shape[0]))
        desc[k] = "WT4_activated_%dH_%dmin"%((80*k)//60, (120*k)%60)


        traces = get_couple_of_measure(elem.measure[i][-2:])
        traces = transform_data(traces.T)
        plot_points(traces, desc[k], ax2, lower_bounds, upper_bounds)
        plt.savefig(save_folder + "/" + name + "/" + desc[k] + ".png")
        k+= 1
        
plt.close("all")

In [15]:
save_david(traces_david, labels_david, desc, name)

3
3
3


# WT222 4H 8H

In [17]:

labels_david = []
traces_david = []
desc = {}
k = 0
name = "WT222_4H"

for i in range(3):
        ax2, lower_bounds, upper_bounds = prepare_plot(database2)


        folder = "../FOLDER_2022_03_16_WT222_SC1/"
        elem = activation_experiment(folder)
        traces, labels, times = elem.get_traces_and_times(elem.measure[i][-2:])
        traces_david.append(traces)
        labels_david.append(np.array([k]*traces.shape[0]))
        desc[k] = name + "_%dH_%dmin"%((80*k)//60, (240*k)%60)


        traces = get_couple_of_measure(elem.measure[i][-2:])
        traces = transform_data(traces.T)
        plot_points(traces, desc[k], ax2, lower_bounds, upper_bounds)
        plt.savefig(save_folder + "/" + name + "/" + desc[k] + ".png")
        k+= 1
        
plt.close("all")

In [18]:
save_david(traces_david, labels_david, desc, name)

3
3
3


## CC124 4H-8H

In [19]:

labels_david = []
traces_david = []
desc = {}
k = 0
name = "CC124_4H"

for i in range(3):
        ax2, lower_bounds, upper_bounds = prepare_plot(database2)


        folder = "../FOLDER_2022_03_30_4H_ENS_CC_124_sync/"
        elem = activation_experiment(folder)
        traces, labels, times = elem.get_traces_and_times(elem.measure[i][-2:])
        traces_david.append(traces)
        labels_david.append(np.array([k]*traces.shape[0]))
        desc[k] = name + "_%dH_%dmin"%((80*k)//60, (240*k)%60)


        traces = get_couple_of_measure(elem.measure[i][-2:])
        traces = transform_data(traces.T)
        plot_points(traces, desc[k], ax2, lower_bounds, upper_bounds)
        plt.savefig(save_folder + "/" + name + "/" + desc[k] + ".png")
        k+= 1
        
plt.close("all")

In [20]:
save_david(traces_david, labels_david, desc, name)

3
3
3


## MIX

In [23]:

labels_david = []
traces_david = []
desc = {}
k = 0
name = "Mix_stt7_wt4_npq4"

for i in range(16):
        ax2, lower_bounds, upper_bounds = prepare_plot(database2)


        folder = "../FOLDER_2022_02_26_activation_4H_ENS_mix/"
        elem = activation_experiment(folder)
        traces, labels, times = elem.get_traces_and_times(elem.measure[i][-2:])
        traces_david.append(traces)
        labels_david.append(np.array([k]*traces.shape[0]))
        desc[k] = name + "_%dH_%dmin"%((80*k)//60, (240*k)%60)


        traces = get_couple_of_measure(elem.measure[i][-2:])
        traces = transform_data(traces.T)
        plot_points(traces, desc[k], ax2, lower_bounds, upper_bounds)
        plt.savefig(save_folder + "/" + name + "/" + desc[k] + ".png")
        k+= 1
        
plt.close("all")

In [ ]:
save_david(traces_david, labels_david, desc, name)